## Packages Installation

First, install the `holisticai` package if you haven't already:
```bash
!pip install holisticai[all]
```
Then, import the necessary libraries.

# Setup Logging

In [1]:
import logging
import sys
import warnings

warnings.filterwarnings("ignore")

logger = logging.getLogger()
logger.setLevel(logging.INFO)
console_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

In [2]:
import pandas as pd
from holisticai.bias.metrics import classification_bias_metrics
from holisticai.datasets import load_dataset
from holisticai.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

## Dataset loading

In [3]:
dataset = load_dataset('adult', protected_attribute='sex')
train_test = dataset.train_test_split(test_size=0.2, random_state=42)

train = train_test['train']
test = train_test['test']

dataset

{"dtype": "Dataset", "attributes": {"Number of Rows": 45222, "Features": ["X , y , group_a , group_b"]}, "metadata": "sex: {'group_a': 'Male', 'group_b': 'Female'}"}

In [4]:
# Define inprocessing model
model = LogisticRegression()

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
model.fit(train['X'], train['y'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = model.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

,Value,Reference
Metric,,
Statistical Parity,0.386984,0
Disparate Impact,2.845278,1
Four Fifths Rule,0.351460,1
Cohen D,0.831806,0
2SD Rule,34.481000,0
Equality of Opportunity Difference,0.214329,0
False Positive Rate Difference,0.324442,0
Average Odds Difference,0.269386,0
Accuracy Difference,-0.196806,0


Now that we have a clean dataset we can start defining the training and testing sets.

# 1. Adversarial Debiasing

### Traditional Implementation

In [5]:
# Define inprocessing model
from holisticai.bias.mitigation import AdversarialDebiasing

mitigator = AdversarialDebiasing(features_dim=train['X'].shape[1], batch_size=512, hidden_size=64, 
                                 adversary_loss_weight=3, verbose=1, use_debias=True, seed=42).transform_estimator()

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

2024-07-31 09:13:39,272 - jax._src.xla_bridge - INFO - Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2024-07-31 09:13:39,274 - jax._src.xla_bridge - INFO - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


2024-07-31 09:13:39.603449: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


2024-07-31 09:13:43,452 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 1/710: Classifier Loss = 0.669206, Adversarial Loss = 0.760739
2024-07-31 09:13:51,927 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 101/710: Classifier Loss = 0.380915, Adversarial Loss = 0.621280
2024-07-31 09:13:59,806 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 201/710: Classifier Loss = 0.365841, Adversarial Loss = 0.620069
2024-07-31 09:14:10,617 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 301/710: Classifier Loss = 0.363159, Adversarial Loss = 0.619218
2024-07-31 09:14:19,648 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 401/710: Classifier Loss = 0.363933, Adversarial Loss = 0.622712
2024-07-31 09:14:31,858 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO 

,Value,Reference
Metric,,
Statistical Parity,0.015605,0
Disparate Impact,1.109120,1
Four Fifths Rule,0.901616,1
Cohen D,0.043291,0
2SD Rule,1.925148,0
Equality of Opportunity Difference,-0.263542,0
False Positive Rate Difference,-0.033950,0
Average Odds Difference,-0.148746,0
Accuracy Difference,-0.101258,0


### Pipeline Implementation

In [6]:
mitigator = AdversarialDebiasing(features_dim=train['X'].shape[1], batch_size=512, hidden_size=64, 
                                 adversary_loss_weight=3, verbose=1, use_debias=True, seed=42).transform_estimator()

# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

2024-07-31 09:14:55,694 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 1/710: Classifier Loss = 0.669206, Adversarial Loss = 0.760739
2024-07-31 09:15:05,907 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 101/710: Classifier Loss = 0.380915, Adversarial Loss = 0.621280
2024-07-31 09:15:14,353 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 201/710: Classifier Loss = 0.365841, Adversarial Loss = 0.620069
2024-07-31 09:15:21,832 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 301/710: Classifier Loss = 0.363159, Adversarial Loss = 0.619218
2024-07-31 09:15:29,371 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO - Step 401/710: Classifier Loss = 0.363933, Adversarial Loss = 0.622712
2024-07-31 09:15:36,823 - holisticai.bias.mitigation.inprocessing.adversarial_debiasing.transformer - INFO 

,Value,Reference
Metric,,
Statistical Parity,0.015605,0
Disparate Impact,1.109120,1
Four Fifths Rule,0.901616,1
Cohen D,0.043291,0
2SD Rule,1.925148,0
Equality of Opportunity Difference,-0.263542,0
False Positive Rate Difference,-0.033950,0
Average Odds Difference,-0.148746,0
Accuracy Difference,-0.101258,0


### Comparison

In [7]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                    0.015605  0.015605         0
Disparate Impact                      1.109120  1.109120         1
Four Fifths Rule                      0.901616  0.901616         1
Cohen D                               0.043291  0.043291         0
2SD Rule                              1.925148  1.925148         0
Equality of Opportunity Difference   -0.263542 -0.263542         0
False Positive Rate Difference       -0.033950 -0.033950         0
Average Odds Difference              -0.148746 -0.148746         0
Accuracy Difference                  -0.101258 -0.101258         0

# 2. Exponentiated Gradient

### Traditional Implementation

In [8]:
# Define inprocessing model
from holisticai.bias.mitigation import ExponentiatedGradientReduction

model = LogisticRegression()
mitigator = ExponentiatedGradientReduction(constraints="DemographicParity").transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

,Value,Reference
Metric,,
Statistical Parity,-0.006845,0
Disparate Impact,0.956970,1
Four Fifths Rule,0.956970,1
Cohen D,-0.018943,0
2SD Rule,-0.842531,0
Equality of Opportunity Difference,-0.324629,0
False Positive Rate Difference,-0.048422,0
Average Odds Difference,-0.186526,0
Accuracy Difference,-0.098872,0


### Pipeline Implementation

In [9]:
mitigator = ExponentiatedGradientReduction(constraints="DemographicParity").transform_estimator(model)

# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

,Value,Reference
Metric,,
Statistical Parity,-0.006845,0
Disparate Impact,0.956970,1
Four Fifths Rule,0.956970,1
Cohen D,-0.018943,0
2SD Rule,-0.842531,0
Equality of Opportunity Difference,-0.324629,0
False Positive Rate Difference,-0.048422,0
Average Odds Difference,-0.186526,0
Accuracy Difference,-0.098872,0


### Comparison

In [10]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                   -0.006845 -0.006845         0
Disparate Impact                      0.956970  0.956970         1
Four Fifths Rule                      0.956970  0.956970         1
Cohen D                              -0.018943 -0.018943         0
2SD Rule                             -0.842531 -0.842531         0
Equality of Opportunity Difference   -0.324629 -0.324629         0
False Positive Rate Difference       -0.048422 -0.048422         0
Average Odds Difference              -0.186526 -0.186526         0
Accuracy Difference                  -0.098872 -0.098872         0

# 3. Grid Search Reduction

### Traditional Implementation

In [11]:
# Define inprocessing model
from holisticai.bias.mitigation import GridSearchReduction

model = LogisticRegression()
mitigator = GridSearchReduction(constraints="DemographicParity", loss='Square', min_val=-0.1, max_val=1.3, 
                                         grid_size=20).transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

,Value,Reference
Metric,,
Statistical Parity,0.008389,0
Disparate Impact,1.055605,1
Four Fifths Rule,0.947324,1
Cohen D,0.023088,0
2SD Rule,1.026893,0
Equality of Opportunity Difference,-0.292173,0
False Positive Rate Difference,-0.037709,0
Average Odds Difference,-0.164941,0
Accuracy Difference,-0.101181,0


### Pipeline Implementation

In [12]:
mitigator = GridSearchReduction(constraints="DemographicParity", loss='Square', min_val=-0.1, max_val=1.3, 
                                         grid_size=20).transform_estimator(model)
# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

,Value,Reference
Metric,,
Statistical Parity,0.008389,0
Disparate Impact,1.055605,1
Four Fifths Rule,0.947324,1
Cohen D,0.023088,0
2SD Rule,1.026893,0
Equality of Opportunity Difference,-0.292173,0
False Positive Rate Difference,-0.037709,0
Average Odds Difference,-0.164941,0
Accuracy Difference,-0.101181,0


### Comparison

In [13]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                    0.008389  0.008389         0
Disparate Impact                      1.055605  1.055605         1
Four Fifths Rule                      0.947324  0.947324         1
Cohen D                               0.023088  0.023088         0
2SD Rule                              1.026893  1.026893         0
Equality of Opportunity Difference   -0.292173 -0.292173         0
False Positive Rate Difference       -0.037709 -0.037709         0
Average Odds Difference              -0.164941 -0.164941         0
Accuracy Difference                  -0.101181 -0.101181         0

# 4. Meta Fair Classifier

### Traditional Implementation

In [14]:
# Define inprocessing model
from holisticai.bias.mitigation import MetaFairClassifier

model = LogisticRegression()
mitigator = MetaFairClassifier(constraint="StatisticalRate", verbose=1, seed=42).transform_estimator(_)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test)

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

[elapsed time: 00:00:06 | iter:8/8 | accuracy:0.6892 | gamma:0.6733]


,Value,Reference
Metric,,
Statistical Parity,-0.078948,0
Disparate Impact,0.850056,1
Four Fifths Rule,0.850056,1
Cohen D,-0.158559,0
2SD Rule,-7.033260,0
Equality of Opportunity Difference,-0.067910,0
False Positive Rate Difference,-0.194712,0
Average Odds Difference,-0.131311,0
Accuracy Difference,0.184129,0


### Pipeline Implementation

In [15]:
mitigator = MetaFairClassifier(constraint="StatisticalRate", verbose=1, seed=42).transform_estimator(_)
# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

[elapsed time: 00:00:03 | iter:8/8 | accuracy:0.6892 | gamma:0.6733]


,Value,Reference
Metric,,
Statistical Parity,-0.078948,0
Disparate Impact,0.850056,1
Four Fifths Rule,0.850056,1
Cohen D,-0.158559,0
2SD Rule,-7.033260,0
Equality of Opportunity Difference,-0.067910,0
False Positive Rate Difference,-0.194712,0
Average Odds Difference,-0.131311,0
Accuracy Difference,0.184129,0


### Comparison

In [16]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional  Pipeline          
                                         Value     Value Reference
Metric                                                            
Statistical Parity                   -0.078948 -0.078948         0
Disparate Impact                      0.850056  0.850056         1
Four Fifths Rule                      0.850056  0.850056         1
Cohen D                              -0.158559 -0.158559         0
2SD Rule                             -7.033260 -7.033260         0
Equality of Opportunity Difference   -0.067910 -0.067910         0
False Positive Rate Difference       -0.194712 -0.194712         0
Average Odds Difference              -0.131311 -0.131311         0
Accuracy Difference                   0.184129  0.184129         0

# 5. Prejudice Remover

### Traditional Implementation

In [17]:
# Define inprocessing model
from holisticai.bias.mitigation import PrejudiceRemover

model = LogisticRegression()
mitigator = PrejudiceRemover(maxiter=100, fit_intercept=True, verbose=1, print_interval=1).transform_estimator(model)

# Standardize data and fit model
scaler = StandardScaler()
X_train = scaler.fit_transform(train['X'])
mitigator.fit(X_train, train['y'], train['group_a'], train['group_b'])

# Standardize data and predict
X_test = scaler.transform(test['X'])
y_pred = mitigator.predict(X_test, test['group_a'], test['group_b'])

# Evaluate bias metrics
metrics = classification_bias_metrics(test['group_a'], test['group_b'], y_pred, test['y'], metric_type='both')
metrics

[elapsed time: 00:00:01 | iter:5/100 | loss:13018.0391]
[elapsed time: 00:00:07 | Best Loss : 13018.0391]


,Value,Reference
Metric,,
Statistical Parity,0.123365,0
Disparate Impact,2.226515,1
Four Fifths Rule,0.449132,1
Cohen D,0.321904,0
2SD Rule,14.158636,0
Equality of Opportunity Difference,-0.001545,0
False Positive Rate Difference,0.036702,0
Average Odds Difference,0.017578,0
Accuracy Difference,-0.103027,0


### Pipeline Implementation

In [18]:
mitigator = PrejudiceRemover(maxiter=100, fit_intercept=True, verbose=1, print_interval=1).transform_estimator(model)

# set up the pipeline
pipeline = Pipeline(steps=[('scalar', StandardScaler()), ('bm_inprocessing', mitigator)])
pipeline.fit(train['X'], train['y'], bm__group_a=train['group_a'], bm__group_b=train['group_b'])

# make predictions
y_pred_pipeline = pipeline.predict(test['X'], bm__group_a=test['group_a'], bm__group_b=test['group_b'])

# compute bias metrics
metrics_pipeline = classification_bias_metrics(test['group_a'], test['group_b'], y_pred_pipeline, test['y'], metric_type='both')
metrics_pipeline

[elapsed time: 00:00:01 | iter:5/100 | loss:13018.0391]
[elapsed time: 00:00:07 | Best Loss : 13018.0391]


,Value,Reference
Metric,,
Statistical Parity,0.123365,0
Disparate Impact,2.226515,1
Four Fifths Rule,0.449132,1
Cohen D,0.321904,0
2SD Rule,14.158636,0
Equality of Opportunity Difference,-0.001545,0
False Positive Rate Difference,0.036702,0
Average Odds Difference,0.017578,0
Accuracy Difference,-0.103027,0


### Comparison

In [19]:
pd.concat([metrics['Value'], metrics_pipeline], axis=1, keys=['Traditional', 'Pipeline'])

Traditional   Pipeline          
                                         Value      Value Reference
Metric                                                             
Statistical Parity                    0.123365   0.123365         0
Disparate Impact                      2.226515   2.226515         1
Four Fifths Rule                      0.449132   0.449132         1
Cohen D                               0.321904   0.321904         0
2SD Rule                             14.158636  14.158636         0
Equality of Opportunity Difference   -0.001545  -0.001545         0
False Positive Rate Difference        0.036702   0.036702         0
Average Odds Difference               0.017578   0.017578         0
Accuracy Difference                  -0.103027  -0.103027         0